# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0  wailua homesteads  22.0669 -159.3780     22.32        78   
1        1          jamestown  42.0970  -79.2353     19.88        99   
2        2           hermanus -34.4187   19.2345     16.56        57   
3        3     puerto natales -51.7236  -72.4875      1.25        98   
4        4             kodiak  57.7900 -152.4072      8.73        81   

   Cloudiness  Wind Speed Country        Date  
0          20        4.63      US  1722507095  
1         100        2.06      US  1722507067  
2           0        0.93      ZA  1722507097  
3         100        0.56      CL  1722507098  
4           0        3.09      US  1722507099

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
# Check the size of the dataset
print(city_data_df.shape)

city_data_df_small = city_data_df.sample(n=100)  # Adjust the number as needed

# Configure the map plot with the smaller dataset
map_plot_small = city_data_df_small.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='Humidity',
    hover_cols=['City', 'Humidity', 'Max Temp', 'Country'],
    tiles='OSM'
)

# Display the smaller map plot
map_plot_small

(604, 10)


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
min_temp = 20  # Minimum temperature in Celsius
max_temp = 30  # Maximum temperature in Celsius
min_humidity = 40  # Minimum humidity percentage
max_humidity = 60  # Maximum humidity percentage

# Narrow down cities that fit the criteria
filtered_city_data_df = city_data_df[
    (city_data_df['Max Temp'] >= min_temp) &
    (city_data_df['Max Temp'] <= max_temp) &
    (city_data_df['Humidity'] >= min_humidity) &
    (city_data_df['Humidity'] <= max_humidity)
]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df.head()

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
19       19        crane  31.3974 -102.3501     25.95        47           5   
31       31  les avirons -21.2418   55.3394     22.87        53           0   
47       47    ariquemes  -9.9133  -63.0408     21.66        56           0   
54       54       uturoa -16.7333 -151.4333     23.37        59           1   
64       64      kirensk  57.7853  108.1119     21.83        46          95   

    Wind Speed Country        Date  
19        7.20      US  1722507115  
31        3.60      RE  1722507128  
47        0.58      BR  1722507145  
54        8.09      PF  1722507152  
64        2.06      RU  1722507163

### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Create a copy of the DataFrame and select relevant columns
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name"
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

City Country      Lat       Lng  Humidity Hotel Name
19        crane      US  31.3974 -102.3501        47           
31  les avirons      RE -21.2418   55.3394        53           
47    ariquemes      BR  -9.9133  -63.0408        56           
54       uturoa      PF -16.7333 -151.4333        59           
64      kirensk      RU  57.7853  108.1119        46

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000  # Search within 5 km radius, adjust as necessary
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1  # Limit to one result per search
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
crane - nearest hotel: No hotel found
les avirons - nearest hotel: Floralys
ariquemes - nearest hotel: Aquarius
uturoa - nearest hotel: Hawaiki Nui hotel
kirensk - nearest hotel: Ивушка
kaga bandoro - nearest hotel: No hotel found
vorgashor - nearest hotel: No hotel found
ceeldheer - nearest hotel: No hotel found
tete - nearest hotel: Zambezi Hotel
alupka - nearest hotel: Сон у моря
antsiranana - nearest hotel: Central
borskoye - nearest hotel: Элегия
barguzin - nearest hotel: No hotel found
mossingen - nearest hotel: Fischers Hotel Garni
tolanaro - nearest hotel: Hôtel Mahavokey
ngong - nearest hotel: Maksen Hotel
sinop - nearest hotel: No hotel found
tlahualilo de zaragoza - nearest hotel: No hotel found
fuerte olimpo - nearest hotel: No hotel found
usinsk - nearest hotel: общежитие Лукойл-Коми
sonoita - nearest hotel: No hotel found
pirane - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
barra do corda - nearest hotel: Hotel Belcho
megi

City Country      Lat       Lng  Humidity         Hotel Name
19        crane      US  31.3974 -102.3501        47     No hotel found
31  les avirons      RE -21.2418   55.3394        53           Floralys
47    ariquemes      BR  -9.9133  -63.0408        56           Aquarius
54       uturoa      PF -16.7333 -151.4333        59  Hawaiki Nui hotel
64      kirensk      RU  57.7853  108.1119        46             Ивушка

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
hvplot.extension('bokeh')

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    tiles='OSM'
)

# Display the map
map_plot